# Portfolio Optimisation with Modern Portfolio Theory

In [ ]:

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize


In [ ]:

# Download historical data for a few ETFs
tickers = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN']
data = yf.download(tickers, start='2019-01-01', end='2023-01-01')['Adj Close']
returns = data.pct_change().dropna()
mean_returns = returns.mean()
cov_matrix = returns.cov()


In [ ]:

# Portfolio performance functions
def portfolio_performance(weights, mean_returns, cov_matrix):
    returns = np.dot(weights, mean_returns)
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return returns, std

def negative_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate=0.01):
    p_return, p_std = portfolio_performance(weights, mean_returns, cov_matrix)
    return - (p_return - risk_free_rate) / p_std


In [ ]:

# Optimise for maximum Sharpe ratio
num_assets = len(tickers)
initial_guess = num_assets * [1. / num_assets]
bounds = tuple((0, 1) for _ in range(num_assets))
constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}

opt_result = minimize(negative_sharpe_ratio, initial_guess,
                      args=(mean_returns, cov_matrix),
                      method='SLSQP', bounds=bounds, constraints=constraints)

opt_weights = opt_result.x


In [ ]:

# Calculate efficient frontier
def efficient_frontier(mean_returns, cov_matrix, num_points=100):
    results = {'Returns': [], 'Volatility': []}
    for r in np.linspace(0.05, 0.5, num_points):
        constraints = (
            {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
            {'type': 'eq', 'fun': lambda x: np.dot(x, mean_returns) - r}
        )
        res = minimize(lambda x: np.sqrt(np.dot(x.T, np.dot(cov_matrix, x))),
                       initial_guess, method='SLSQP', bounds=bounds, constraints=constraints)
        if res.success:
            results['Returns'].append(r)
            results['Volatility'].append(res.fun)
    return results

frontier = efficient_frontier(mean_returns, cov_matrix)


In [ ]:

# Plot efficient frontier
plt.figure(figsize=(10, 6))
plt.plot(frontier['Volatility'], frontier['Returns'], 'b--', label='Efficient Frontier')
opt_return, opt_std = portfolio_performance(opt_weights, mean_returns, cov_matrix)
plt.scatter(opt_std, opt_return, color='red', label='Max Sharpe Ratio')
plt.title('Efficient Frontier with Optimal Portfolio')
plt.xlabel('Volatility (Standard Deviation)')
plt.ylabel('Expected Return')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# Show optimal weights
opt_df = pd.DataFrame({'Ticker': tickers, 'Optimal Weight': opt_weights})
opt_df
